**Общая информация**

**Срок сдачи:** 29 марта 2021, 08:30 

**Штраф за опоздание:** по 1 баллу за 24 часа задержки. Через 5 дней домашнее задание сгорает.

При отправлении ДЗ указывайте фамилию в названии файла
Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:

[ML0220, Задание 1] Фамилия Имя.

Используйте данный Ipython Notebook при оформлении домашнего задания.


**Штрафные баллы:**

1. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw1.ipynb) -0.5 баллов
2. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -0.5 баллов


In [41]:
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn import datasets
from sklearn.base import BaseEstimator
from sklearn.datasets import fetch_openml, fetch_20newsgroups

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from scipy.spatial.distance import cdist
from sklearn.neighbors import KDTree
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate

##### Задание 1 (1 балл)
Реализовать KNN в классе MyKNeighborsClassifier (обязательное условие: точность не ниже sklearn реализации)
Разберитесь самостоятельно, какая мера расстояния используется в KNeighborsClassifier дефолтно и реализуйте свой алгоритм именно с этой мерой. 
Для подсчета расстояний можно использовать функции [отсюда](https://docs.scipy.org/doc/scipy/reference/spatial.distance.html)

In [15]:
class MyKNeighborsClassifier(BaseEstimator):
    
    def __init__(self, n_neighbors, algorithm='brute'):
            self.n_neighbors=n_neighbors
            self.algorithm=algorithm
        #self.weights = _check_weights(weights)
    
    def fit(self, X, y):
        self.X = X
        self.y = y
    
    def predict(self, X_test):

        if self.algorithm == 'brute':
            neighbors = cdist(X_test, self.X, metric='minkowski')
            
            y_pred = np.array([
                np.argmax(np.bincount(y_train[neighbor[:self.n_neighbors]]))
                for neighbor in np.argsort(neighbors, axis=1)
            ])

            
        elif self.algorithm=='kd_tree':
            tree = KDTree(self.X)
            dist, ind = tree.query(X_test, k=self.n_neighbors) 
            y_pred = np.array([
                np.argmax(np.bincount(y_train[neighbor]))
                for neighbor in ind
            ])
        return y_pred

**IRIS**

В библиотеке scikit-learn есть несколько датасетов из коробки. Один из них [Ирисы Фишера](https://ru.wikipedia.org/wiki/%D0%98%D1%80%D0%B8%D1%81%D1%8B_%D0%A4%D0%B8%D1%88%D0%B5%D1%80%D0%B0)

In [16]:
iris = datasets.load_iris()

In [17]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [18]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='brute')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='brute')

In [19]:
clf.fit(X_train, y_train)
my_clf.fit(X_train, y_train)

In [20]:
my_clf.predict(X_test)

array([1, 0, 2, 2, 0, 1, 0, 2, 0, 2, 1, 0, 2, 2, 1])

In [21]:
sklearn_pred = clf.predict(X_test)
my_clf_pred = my_clf.predict(X_test)
assert abs(accuracy_score(y_test, my_clf_pred) -  accuracy_score(y_test, sklearn_pred ) )<0.005, "Score must be simillar"

**Задание 2 (0.5 балла)**

Давайте попробуем добиться скорости работы на fit, predict сравнимой со sklearn для iris. Допускается замедление не более чем в 2 раза. 
Для этого используем numpy. 

In [22]:
%time clf.fit(X_train, y_train)

CPU times: user 701 µs, sys: 400 µs, total: 1.1 ms
Wall time: 735 µs


KNeighborsClassifier(algorithm='brute', n_neighbors=2)

In [23]:
%time my_clf.fit(X_train, y_train)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.87 µs


In [24]:
%time clf.predict(X_test)

CPU times: user 1.79 ms, sys: 1.18 ms, total: 2.97 ms
Wall time: 1.89 ms


array([1, 0, 2, 2, 0, 1, 0, 2, 0, 2, 1, 0, 2, 2, 1])

In [25]:
%time my_clf.predict(X_test)

CPU times: user 1.01 ms, sys: 697 µs, total: 1.7 ms
Wall time: 798 µs


array([1, 0, 2, 2, 0, 1, 0, 2, 0, 2, 1, 0, 2, 2, 1])

###### Задание 3 (1 балл)
Добавьте algorithm='kd_tree' в реализацию KNN (использовать KDTree из sklearn.neighbors). Необходимо добиться скорости работы на fit,  predict сравнимой со sklearn для iris. Допускается замедление не более чем в 2 раза. 
Для этого используем numpy. Точность не должна уступать значению KNN из sklearn. 

In [26]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')

In [27]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [28]:
%time clf.fit(X_train, y_train)

CPU times: user 726 µs, sys: 232 µs, total: 958 µs
Wall time: 813 µs


KNeighborsClassifier(algorithm='kd_tree', n_neighbors=2)

In [29]:
%time my_clf.fit(X_train, y_train)

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 7.15 µs


In [30]:
%time clf.predict(X_test)

CPU times: user 2.08 ms, sys: 1.3 ms, total: 3.38 ms
Wall time: 2.26 ms


array([2, 0, 1, 0, 2, 2, 2, 0, 1, 2, 0, 0, 1, 1, 1])

In [31]:
%time my_clf.predict(X_test)

CPU times: user 648 µs, sys: 305 µs, total: 953 µs
Wall time: 749 µs


array([2, 0, 1, 0, 2, 2, 2, 0, 1, 2, 0, 0, 1, 1, 1])

In [32]:
sklearn_pred = clf.predict(X_test)
my_clf_pred = my_clf.predict(X_test)
assert abs( accuracy_score(y_test, my_clf_pred) -  accuracy_score(y_test, sklearn_pred ) )<0.005, "Score must be simillar"

**Задание 4 (2.5 балла)**

Рассмотрим новый датасет 20 newsgroups

In [33]:
newsgroups = fetch_20newsgroups(subset='train',remove=['headers','footers', 'quotes'])

In [34]:
data = newsgroups['data']
target = newsgroups['target']

Преобразуйте текстовые данные из data с помощью [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html). Словарь можно ограничить по частотности.

In [74]:
vectorizer = CountVectorizer(max_df=500, min_df=2, max_features=500, stop_words='english', lowercase=True)
X = vectorizer.fit_transform(data)

*Так мы получили векторное представление наших текстов. Значит можно приступать к задаче обучения модели*

Реализуйте разбиение выборки для кросс-валидации на 3 фолдах. Разрешено использовать sklearn.cross_validation

In [38]:
folds = 3
kf = KFold(n_splits=folds, shuffle=True)
kf.get_n_splits(X)

3

In [75]:
model = KNeighborsClassifier()
cv = cross_validate(model,X,target,cv = kf,return_train_score = True)
cv['test_score'].mean()

0.2861071905658699

In [57]:
def change_params(X, target, algorithm='brute',  metric='minkowski'):
    best_model = None
    best_score = 0
    for i in range(1,20):
        model = KNeighborsClassifier(n_neighbors=i, algorithm=algorithm, metric=metric)
        cv = cross_validate(model,X,target,cv = kf,return_train_score = True)
        print(f"n_neighbors = {i}, score_test = {cv['test_score'].mean()}, score_train = {cv['train_score'].mean()}")
        if cv['test_score'].mean()> best_score:
            best_score = cv['test_score'].mean()
            best_model = model
    return model, cv

change_params(X, target)

n_neighbors = 1, score_test = 0.2788577204370501, score_train = 0.92151318093656
n_neighbors = 2, score_test = 0.25676147590226206, score_train = 0.6011583676786869
n_neighbors = 3, score_test = 0.26657249390452464, score_train = 0.5259409269878127
n_neighbors = 4, score_test = 0.27824102546653084, score_train = 0.5102975241076064
n_neighbors = 5, score_test = 0.2836308729556805, score_train = 0.49787876359984656
n_neighbors = 6, score_test = 0.2912314814580473, score_train = 0.47724065515347763
n_neighbors = 7, score_test = 0.2921164982636648, score_train = 0.4724672858550955
n_neighbors = 8, score_test = 0.2908796400578581, score_train = 0.45664643896341434
n_neighbors = 9, score_test = 0.29406106995593145, score_train = 0.4471453152665752
n_neighbors = 10, score_test = 0.2900838842015759, score_train = 0.4425937041078804
n_neighbors = 11, score_test = 0.2907893245685596, score_train = 0.43375446227685277
n_neighbors = 12, score_test = 0.2930878936093847, score_train = 0.428495500835

(KNeighborsClassifier(algorithm='brute', n_neighbors=19),
 {'fit_time': array([0.00167894, 0.00169611, 0.00160003]),
  'score_time': array([0.79786801, 0.81522822, 0.7649219 ]),
  'test_score': array([0.29825027, 0.27870591, 0.29382127]),
  'train_score': array([0.39485548, 0.39467056, 0.40262495])})

Как изменится качество на валидации, если:

1. Используется косинусная метрика вместо евклидовой.
2. К текстам применяется TfIdf векторизацию( sklearn.feature_extraction.text.TfidfVectorizer)

Сравните модели, выберите лучшую.

In [58]:
change_params(X, target, metric='cosine')

n_neighbors = 1, score_test = 0.3206637387013073, score_train = 0.9214690543069729
n_neighbors = 2, score_test = 0.31297527530288966, score_train = 0.627364382566617
n_neighbors = 3, score_test = 0.3298562572980024, score_train = 0.5695159239008235
n_neighbors = 4, score_test = 0.3451484225160124, score_train = 0.5551087228942465
n_neighbors = 5, score_test = 0.35761034307793876, score_train = 0.5411879636335635
n_neighbors = 6, score_test = 0.3640627450340776, score_train = 0.5353101922764417
n_neighbors = 7, score_test = 0.37060239259651073, score_train = 0.5224502064881346
n_neighbors = 8, score_test = 0.37899863978405274, score_train = 0.5140533559447301
n_neighbors = 9, score_test = 0.3848338077825799, score_train = 0.5091038844219099
n_neighbors = 10, score_test = 0.3899600905835768, score_train = 0.5040219802763399
n_neighbors = 11, score_test = 0.39031277561573646, score_train = 0.4965527557440526
n_neighbors = 12, score_test = 0.39393579998198375, score_train = 0.4935918829220

(KNeighborsClassifier(algorithm='brute', metric='cosine', n_neighbors=19),
 {'fit_time': array([0.00182986, 0.0016911 , 0.00174093]),
  'score_time': array([0.69190502, 0.73783898, 0.74759507]),
  'test_score': array([0.40058324, 0.41262265, 0.4089101 ]),
  'train_score': array([0.48952532, 0.48031287, 0.48985815])})

In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=2000, min_df=20, max_df=1000,\
                             stop_words='english', token_pattern='\w*[a-zA-Z]\w+')
X = vectorizer.fit_transform(data)

folds = 3
kf = KFold(n_splits=folds, shuffle=True)
kf.get_n_splits(X)
change_params(X, target, metric='cosine')

n_neighbors = 1, score_test = 0.4748111412662672, score_train = 0.9662365366172687
n_neighbors = 2, score_test = 0.4510338897273653, score_train = 0.7123032344016759
n_neighbors = 3, score_test = 0.478168023180007, score_train = 0.6886600796290255
n_neighbors = 4, score_test = 0.5018573495201937, score_train = 0.6767283233630353
n_neighbors = 5, score_test = 0.5190911102843518, score_train = 0.6722202236040636
n_neighbors = 6, score_test = 0.5196217547938683, score_train = 0.6612603078687583
n_neighbors = 7, score_test = 0.5345587978206924, score_train = 0.6576807305987057
n_neighbors = 8, score_test = 0.538713779012855, score_train = 0.6524219859528607
n_neighbors = 9, score_test = 0.5433975768452645, score_train = 0.6478257149653633
n_neighbors = 10, score_test = 0.5479052665038081, score_train = 0.6484002634871305
n_neighbors = 11, score_test = 0.5498497444591892, score_train = 0.6422130256490268
n_neighbors = 12, score_test = 0.552235277895652, score_train = 0.6440250123634836
n_ne

(KNeighborsClassifier(algorithm='brute', metric='cosine', n_neighbors=19),
 {'fit_time': array([0.0028069 , 0.0026679 , 0.00342512]),
  'score_time': array([0.96459413, 1.0525341 , 0.87567616]),
  'test_score': array([0.55805938, 0.57915672, 0.56483691]),
  'train_score': array([0.64173959, 0.62720403, 0.63568872])})

Загрузим  теперь test  часть нашей выборки и преобразуем её аналогично с train частью. Не забудьте, что наборы слов в train и test части могут отличаться.

In [63]:
vectorizer = TfidfVectorizer(max_features=2000, min_df=20, max_df=1000,\
                             stop_words='english', token_pattern='\w*[a-zA-Z]\w+')
X_best = vectorizer.fit_transform(data)

best_k=10 # По заданиию не должно превышать 10
knn_best = KNeighborsClassifier(n_neighbors=best_k, algorithm='brute', metric='cosine')
knn_best.fit(X_best.toarray(), target)

KNeighborsClassifier(algorithm='brute', metric='cosine', n_neighbors=10)

In [64]:
newsgroups = fetch_20newsgroups(subset='test',remove=['headers','footers', 'quotes'])

Оценим точность вашей лучшей модели на test части датасета. Отличается ли оно от кросс-валидации? Попробуйте сделать выводы, почему отличается качество.

In [72]:
data_test = newsgroups['data']
target_test = newsgroups['target']
X_test = vectorizer.transform(data_test)

knn_pred_test = knn_best.predict(X_test.toarray())
knn_best.score(X_test.toarray(),target_test)

0.4782262347318109

Выводы: Точность хоть и высокая, но не идеальная. Модель сильно переобучена - это видно по качеству на трейне и на тесте. Возможно, в тестовой части присутствуют данные, которых нет в трейне и это так же плохо влияет на качество модели. Метод KNN для классификации тектов - не самая лучшая модель